<a href="https://colab.research.google.com/github/ayalarodrigues/LLL-RAG-Chatbot-Suporte/blob/main/LLM_RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação das bibliotecas


In [ ]:
!pip install langchain langchain-groq langchain_community langchain-huggingface --q
!pip install faiss-cpu sentence-transformers PyMuPDF --q

!pip install -U sentence-transformers
!pip install -U transformers


#langchain-grop: acesso aos modelos via API
#faiss-cpu: onde serão armazenados os documentos(pode ser cpu ou gpu)
#sentence-transformers: conversão dos textos para números
#PyMuPDF: leitura e processamento de texto


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

In [ ]:
import nbformat
from google.colab import files, _message

# Lê o conteúdo do notebook atual diretamente da memória
nb_data = _message.blocking_request('get_ipynb')['ipynb']

# Converte para estrutura nbformat
nb = nbformat.from_dict(nb_data)

# Remove os metadados 'widgets' se existirem
if 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']
    print("Metadados 'widgets' removidos.")
else:
    print("Nenhum metadado 'widgets' encontrado.")

# Nome do novo arquivo corrigido
clean_name = "notebook_corrigido.ipynb"

# Salva no ambiente do Colab
with open(clean_name, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

# Oferece para download (ou você pode subir manualmente para o GitHub)
files.download(clean_name)


Metadados 'widgets' removidos.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Importações

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
import getpass

Carregamento da LLM

In [ ]:
os.environ["GROQ_API_KEY"] = getpass.getpass()
#getpass não exibe a chave da API no CF


··········


In [ ]:
def load_llm(id_model, temperature):
  llm = ChatGroq(
      model = id_model,
      temperature = temperature,
      max_tokens = None,
      timeout = None,
      max_retries = 2
  )
  return llm

In [ ]:
#teste
id_model = "llama3-70b-8192" # @param {type: "string"}
temperature = 0.7 # @param {type: "slider", min: 0.1, max: 1.5, step: 0.1}

llm = load_llm(id_model, temperature)

In [ ]:
#teste
prompt = "Como alterar minha senha?" # @param {type: "string"}

template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente virtual prestativo e está respondendo perguntas gerais"),
    ("human", "{prompt}")
])

chain = template | llm | StrOutputParser()

res = chain.invoke({"prompt": prompt})
res

'Uma pergunta muito comum e importante!\n\nA forma de alterar sua senha varia dependendo do sistema, aplicativo ou plataforma que você esteja usando. Aqui estão as etapas gerais para alterar sua senha em alguns casos comuns:\n\n**Senha de login do computador ou laptop:**\n\n1. Clique no ícone do seu perfil ou nome de usuário no canto superior direito da tela de login.\n2. Selecione "Alterar senha" ou "Editar perfil".\n3. Digite sua senha atual e, em seguida, digite a nova senha duas vezes para confirmar.\n4. Clique em "Salvar" ou "Alterar senha".\n\n**Senha de aplicativos ou serviços online (ex: email, redes sociais, etc.):**\n\n1. Faça login na sua conta normalmente.\n2. Clique no ícone de engrenagem ou em "Configurações" no canto superior direito da tela.\n3. Selecione "Segurança" ou "Conta" e, em seguida, "Alterar senha".\n4. Digite sua senha atual e, em seguida, digite a nova senha duas vezes para confirmar.\n5. Clique em "Salvar" ou "Alterar senha".\n\n**Senha de aplicativos móvei

In [ ]:
def show_res(res):
  from IPython.display import display, Markdown
  if "</think>" in res:
    res = res.split("</think>")[-1].strip()  #o modelo pode retornar o passo a passo necessário para resolução do problema
  else:
    res = res.strip()
  display(Markdown(res))


show_res(res)

 #a tag </think> serve para delimitar pensamentos intermediários, o que permite visualizar o "raciocínio" utilizado na resposta
 #optei por remover a tag por não ser de interesse da persona

Uma pergunta muito comum e importante!

A forma de alterar sua senha varia dependendo do sistema, aplicativo ou plataforma que você esteja usando. Aqui estão as etapas gerais para alterar sua senha em alguns casos comuns:

**Senha de login do computador ou laptop:**

1. Clique no ícone do seu perfil ou nome de usuário no canto superior direito da tela de login.
2. Selecione "Alterar senha" ou "Editar perfil".
3. Digite sua senha atual e, em seguida, digite a nova senha duas vezes para confirmar.
4. Clique em "Salvar" ou "Alterar senha".

**Senha de aplicativos ou serviços online (ex: email, redes sociais, etc.):**

1. Faça login na sua conta normalmente.
2. Clique no ícone de engrenagem ou em "Configurações" no canto superior direito da tela.
3. Selecione "Segurança" ou "Conta" e, em seguida, "Alterar senha".
4. Digite sua senha atual e, em seguida, digite a nova senha duas vezes para confirmar.
5. Clique em "Salvar" ou "Alterar senha".

**Senha de aplicativos móveis:**

1. Abra o aplicativo e faça login como de costume.
2. Toque no ícone de engrenagem ou em "Configurações" no canto superior direito da tela.
3. Selecione "Conta" ou "Segurança" e, em seguida, "Alterar senha".
4. Digite sua senha atual e, em seguida, digite a nova senha duas vezes para confirmar.
5. Toque em "Salvar" ou "Alterar senha".

Lembre-se de escolher uma senha forte e única para cada conta, e de não compartilhá-la com ninguém!

Se você tiver alguma dúvida específica sobre como alterar sua senha em um sistema ou aplicativo específico, sinta-se à vontade para perguntar!

Definição do contexto

Há basicamente duas possibilidades para o exemplo de teste: o algoritmo não consegue retornar com exatidão o caminho exato para recuperar a senha ou a informação foi "inventada"(processo de alucinação).

O modelo quer apenas prever a próxima combinação de palavras que faça sentido. Nem sempre poderá ser entendido que o modelo não sabe essa informação com exatidão. O modelo pode apenas sugerir um caminho comum, como o retornado no exemplo, porém, o algoritmo não possui meios de conhecer o caminho exato sem consultar uma fonte que comprove.

Nesse aspecto, é importante que seja fornecido um contexto que tenha as informações exatas para a LLM e assim a dúvida do usuário pode ser respondida.

Para a obtenção do contexto, nesse estudo de caso, será feito o carregamento do documento que posteriormente será processado pelo pipeline de RAG.



In [ ]:
#teste

context = """
Para alterar uma senha no aplicativo, clique no menu 'Minha conta' e selecione 'Alterar senha'.
Para alterar a senha pelo site, acesse 'Configurações' no menu do topo. Em seguida, selecione 'Minha conta' e 'Alterar senha'.
"""

prompt = f"""
Como alterar minha senha?

Contexto: {context}
"""

In [ ]:
#teste

prompt

"\nComo alterar minha senha?\n\nContexto: \nPara alterar uma senha no aplicativo, clique no menu 'Minha conta' e selecione 'Alterar senha'.\nPara alterar a senha pelo site, acesse 'Configurações' no menu do topo. Em seguida, selecione 'Minha conta' e 'Alterar senha'.\n\n"

In [ ]:
#teste

res = chain.invoke({"prompt": prompt})
show_res(res)


Para alterar sua senha, há duas opções dependendo de onde você está acessando a conta:

**No aplicativo:**

1. Clique no menu "Minha conta"
2. Selecione "Alterar senha"

**No site:**

1. Acesse o menu do topo e clique em "Configurações"
2. Selecione "Minha conta"
3. Clique em "Alterar senha"

Siga esses passos para alterar sua senha com segurança!

Prompt para o RAG

In [ ]:
template_rag = """
Pergunta: {input}
Contexto: {context}
"""

In [ ]:
from langchain.prompts import PromptTemplate #é utilizado para prompts personalizados

prompt_rag = PromptTemplate.from_template(template_rag)
print(prompt_rag)


input_variables=['context', 'input'] input_types={} partial_variables={} template='\nPergunta: {input}\nContexto: {context}\n'


Criação dos textos e geração da Chain

In [ ]:
chaing_rag = prompt_rag | llm | StrOutputParser()

input = "Como alterar minha senha?"
res = chaing_rag.invoke({"input": input, "context": context}) # o envio de variáveis deve seguir de acordo com o que foi definido no template do prompt
show_res(res)

Para alterar sua senha, siga os seguintes passos:

**No aplicativo:**

1. Clique no menu "Minha conta".
2. Selecione "Alterar senha".

**No site:**

1. Acesse "Configurações" no menu do topo.
2. Selecione "Minha conta".
3. Clique em "Alterar senha".

Siga as instruções subsequentes para criar uma nova senha segura.

Etapas de Indexação

Para este estudo de caso será feito o uso de funções(Document Loaders) para importar dados na LangChain.

Os documentos serão quebrados para melhor indexação e processamento do modelo.

Na sequência, será feito o embedding(conversão dos textos em representações numéricas) em cada pedaço e por fim os mesmos serão salvos em um banco de dados vetorial.

Carregamento do conteúdo dos documentos

In [ ]:
from google.colab import files

uploaded = files.upload()
file_path =list(uploaded.keys())[0]

print(f"Arquivo carregado: {file_path}")

Saving manual_facesecure.pdf to manual_facesecure.pdf
Arquivo carregado: manual_facesecure.pdf


In [ ]:
#carregar e processar o conteúdo de um arquivo .pdf

from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "manual_facesecure.pdf"

loader = PyMuPDFLoader(file_path) #na chamada da função, o documento é lido e transformado em uma lista de objetos do tipo 'documents', cada um representando uma página do pdf com o texto extraído
doc = loader.load()
doc[0] #cada posição é uma página do documento

Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': '', 'creationdate': 'D:20250728031816', 'source': 'manual_facesecure.pdf', 'file_path': 'manual_facesecure.pdf', 'total_pages': 4, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': 'D:20250728031816', 'page': 0}, page_content='Manual de Atendimento e Uso - FaceSecure\n1. Introdução\nObjetivo do Manual\nEste manual tem como objetivo orientar clientes e atendentes sobre o funcionamento completo do\nsistema de reconhecimento facial FaceSecure, abordando desde o cadastro, funcionalidades e\nserviços até procedimentos de segurança, suporte e resolução de problemas. O documento\ntambém serve como material de consulta para dúvidas frequentes.\nVisão geral da FaceSecure\nO FaceSecure é um sistema de reconhecimento facial que oferece autenticação segura, rápida e\ninteligente para diversos cenários: acesso a sistemas, c

In [ ]:
#informações sobre o pdf
import pprint

pprint.pprint(doc[0].metadata)

{'author': '',
 'creationDate': 'D:20250728031816',
 'creationdate': 'D:20250728031816',
 'creator': '',
 'file_path': 'manual_facesecure.pdf',
 'format': 'PDF 1.3',
 'keywords': '',
 'modDate': '',
 'moddate': '',
 'page': 0,
 'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/',
 'source': 'manual_facesecure.pdf',
 'subject': '',
 'title': '',
 'total_pages': 4,
 'trapped': ''}


In [ ]:
#função para a extração dos textos

def extract_text_pdf(file_path):
  loader = PyMuPDFLoader(file_path)
  doc = loader.load()  #na variável doc há uma lista com cada uma das páginas
  content = "\n".join([page.page_content for page in doc]) #percorre cada pasta extraindo o texto e concatenando em um única variável
  return content

In [ ]:
extract_text_pdf(file_path)

'Manual de Atendimento e Uso - FaceSecure\n1. Introdução\nObjetivo do Manual\nEste manual tem como objetivo orientar clientes e atendentes sobre o funcionamento completo do\nsistema de reconhecimento facial FaceSecure, abordando desde o cadastro, funcionalidades e\nserviços até procedimentos de segurança, suporte e resolução de problemas. O documento\ntambém serve como material de consulta para dúvidas frequentes.\nVisão geral da FaceSecure\nO FaceSecure é um sistema de reconhecimento facial que oferece autenticação segura, rápida e\ninteligente para diversos cenários: acesso a sistemas, controle de entrada em ambientes físicos,\nverificação de identidade em aplicativos e muito mais. A solução é baseada em inteligência artificial\ne biometria facial, com foco em precisão, privacidade e facilidade de uso.\nPúblico-alvo\nA solução é voltada para empresas e usuários finais que buscam uma forma moderna e segura de\nautenticação, substituindo senhas e cartões por reconhecimento facial.\n2. 

In [ ]:
#leitura de todos os arquivos .pdf contidos em uma pasta

docs_path = Path("/content") #pasta padrão do colab
pdf_files = [f for f in docs_path.glob("*.pdf")]

print(pdf_files)

[PosixPath('/content/manual_facesecure.pdf')]


In [ ]:
loaded_documents = [extract_text_pdf(pdf) for pdf in pdf_files] #percorre cada um dos arquivos pdf e faz a extração por meio da função extract_file_pdf
len(pdf_files)

1

**Divisão em pedaços de texto / Split**

Neste estudo de caso, este passo se torna necessário porque os documentos longos podem comprometer o desempenho da recuperação dos dados. Desta forma, o texto será dividido em "pedaços" menores, pois o documento carregado é muito grande para ser passado como contexto pelo prompt.

Dependendo do conteúdo, poderia ser possível caber o conteúdo inteiro na janela de contexto. Por exemplo, no modelo GPT-4 a janela de contexto é de cerca de 8k tokens, o que equivale, aproximadamente, a 32k caracteres. Porém, mesmo assim, as LLMs terão dificuldade em localizar informações dentro de entradas tão longas.

**Portanto, tal divisão do texto é considerada uma boa prática.**


Há dois grandes motivos pelos quais a fragmentação é necessária para qualquer aplicação que envolva bancos de dados vetoriais ou LLMs: para garantir que os modelos de incorporação possam ajustar os dados em suas janelas de contexto e para garantir que os próprios blocos contenham as informações necessárias para a pesquisa.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_sppliter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)

#chunk_size: quantidade total de tokens em cada uma das partes;
#chunk_overlap: sobreposição de textos entre cada uma das partes
#chunk_overlap: quantos caracteres ou tokens do final do final de um chunk(ou parte) serão reutilizados no começo do próximo

chunks = []

for doc in loaded_documents:
  chunks.extend(text_sppliter.split_text(doc))

  print(f"Número de chunks criados:{len(chunks)}") #indica a quantidade de "pedaços" em que o contexto foi "quebrado"

Número de chunks criados:11


**Embeddings** são representações vetoriais que organizam palavras com base em sua similaridade semântica, agrupando termos relacionados em um espaço multidimensional.

Quando uma palavra nova é apresentada, o modelo consegue posicioná-la corretamente nesse espaço, considerando sua relação com outras palavras.

Com o uso do Mecanismo de Atenção, o modelo compreende o contexto em que a palavra aparece, permitindo diferenciar significados e ajustar com precisão sua posição entre os embeddings.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = "sentence-transformers/all-mpnet-base-v2"
#embedding_model = "BAAI/bge-m3"

embeddings = HuggingFaceEmbeddings(model_name = embedding_model)

input_test = "Um teste apenas"

result = embeddings.embed_query(input_test)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(result) #a frase "Um teste apenas" foi convertida para o seguinte embedding, que é valor um conjuntos de len(result) valores

[-0.026895038783550262, -0.011064727790653706, -0.04532000422477722, -0.0013972934102639556, 0.04243597015738487, -0.014201696030795574, 0.023354969918727875, 0.06011558324098587, 0.06152193248271942, 0.006502091884613037, 0.008159181103110313, -0.03056747280061245, 0.002060319297015667, 0.0129615543410182, 0.0042511713691055775, 0.0036631484981626272, -0.026755547150969505, 0.029737001284956932, -0.009770059026777744, -0.04650460556149483, -0.028108958154916763, 0.00016859530296642333, -0.02481103502213955, -0.01182831171900034, 0.0818282887339592, 0.0014993064105510712, 0.013264846988022327, -0.0624217726290226, -0.0012287232093513012, 0.02897718735039234, -0.02952898107469082, -0.02569606713950634, 0.0033771232701838017, -0.028868917375802994, 1.518517706244893e-06, -0.038824815303087234, -0.019842024892568588, -0.01765807531774044, -0.008112371899187565, -0.025130528956651688, 0.02574918232858181, 0.11367510259151459, -0.006133151240646839, -0.017986435443162918, -0.045411463826894

In [ ]:
len(result) #representação numérica do texto

768

In [ ]:
print(result)

[-0.026895038783550262, -0.011064727790653706, -0.04532000422477722, -0.0013972934102639556, 0.04243597015738487, -0.014201696030795574, 0.023354969918727875, 0.06011558324098587, 0.06152193248271942, 0.006502091884613037, 0.008159181103110313, -0.03056747280061245, 0.002060319297015667, 0.0129615543410182, 0.0042511713691055775, 0.0036631484981626272, -0.026755547150969505, 0.029737001284956932, -0.009770059026777744, -0.04650460556149483, -0.028108958154916763, 0.00016859530296642333, -0.02481103502213955, -0.01182831171900034, 0.0818282887339592, 0.0014993064105510712, 0.013264846988022327, -0.0624217726290226, -0.0012287232093513012, 0.02897718735039234, -0.02952898107469082, -0.02569606713950634, 0.0033771232701838017, -0.028868917375802994, 1.518517706244893e-06, -0.038824815303087234, -0.019842024892568588, -0.01765807531774044, -0.008112371899187565, -0.025130528956651688, 0.02574918232858181, 0.11367510259151459, -0.006133151240646839, -0.017986435443162918, -0.045411463826894

#Geração de embeddings e indexação

Armazenamento no banco de dados vetorial

Aqui os embeddings gerados serão armazenados em um banco de dados vetorial.

Todos os textos estão dentro da variável 'chunks'. Tem-se o total de len(chunks) divisões

In [ ]:
len(chunks)

11

In [ ]:
#Aplicação do embeddings
#conversão do texto para representações numéricas, bem como a criação da base de dados
vectorstore = FAISS.from_texts(chunks, embedding=embeddings) #embeddings está fazendo a implementação do modelo

Salvado índice FAISS

In [ ]:
vectorstore.save_local("faiss_index") #os arquivos gerados na pasta são usados para armazenar os dados

In [ ]:
#Para carregar o índice salvo no disco

#db = FAISS.load_local("faiss_index", embeddings)

# Recuperação de dados e geração de textos

Busca os dados mais relevantes solicitados pelo usuário, usando-os na geraçaõ das respostas. A pergunta do usuário é enviada à base de dados, são feitos vários cálculos de similaridade entre a pergunta do usuário e os dados armazenados(a similaridade é calculada por meio dos embeddings). Na sequência, essa mesma questão, juntamente com os embeddings mais similares, é enviada à LLM para gerar a resposta.

O modelo responde combinando a pergunta do usuário com o contexto recuperado.

Configurando o recuperador de texto / retriever

In [ ]:
#classe do langchain que envolve um índice capaz de retornar objetos do tipo documents dada uma consulta em formato de string
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs = {"k": 6})
#similarity é o tipo de pesquisa que será realizada
#k: indica o número de documentos retornados pelo recuperador; aqui é feita a busca de 6 documentos mais similares para que a LLM baseie sua resposta
#o valor pode variar para mais ou menos documentos, dependendo das características do estudo de caso
retriever

#basicamente, o que se está fazendo é a busca de similaridade dentro do documento

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ed0a261a410>, search_kwargs={'k': 6})

Geração dos textos

In [ ]:
prompt_rag = PromptTemplate(
    input_variables = ["context", "input"],
    template = template_rag,
)

prompt_rag


PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nPergunta: {input}\nContexto: {context}\n')

In [ ]:
#criação da chain
from langchain_core.runnables import RunnablePassthrough #classe para criar fluxos ou pipelines

chain_rag = (
    {"context": retriever, "input": RunnablePassthrough()} | prompt_rag | llm | StrOutputParser()
)

In [ ]:
#teste

res = chain_rag.invoke("Quais são as etapas do cadastro?")
show_res(res)


As etapas do cadastro são:

1. Acesse o app ou portal da FaceSecure
2. Informe dados pessoais (nome, CPF, e-mail)
3. Envie foto de documento oficial
4. Grave o vídeo para reconhecimento facial
5. Concorde com os termos de uso e política de privacidade

# Melhorias no prompt

Melhorias no prompt para deixá-lo mais específico através da adesão de algumas regras para garantir que o resultado seja o mais próximo do que o usuário deseja

In [ ]:
system_prompt = """Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os serviços de uma empresa.
Use os seguintes pedaços de contexto recuperado para responder à pergunta.
Se você não sabe a resposta, apenas comente que não sabe dizer com certeza.
Mas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.
Mantenha a resposta concisa.
Responda em português. \n\n"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Pergunta: {input}\n\n Contexto: {context}"),
    ]
)
qa_prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os serviços de uma empresa.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas comente que não sabe dizer com certeza.\nMas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.\nMantenha a resposta concisa.\nResponda em português. \n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Pergunta: {input}\n\n Contexto: {context}'), additional_kwargs={})])

In [ ]:
chain_rag = (
    {"context": retriever, "input": RunnablePassthrough()} | prompt_rag | llm | StrOutputParser()
)

res = chain_rag.invoke("Quais são as etapas do cadastro?")
show_res(res)

De acordo com o contexto, as etapas do cadastro são:

1. Acesse o app ou portal da FaceSecure
2. Informe dados pessoais (nome, CPF, e-mail)
3. Envie foto de documento oficial
4. Grave o vídeo para reconhecimento facial
5. Concorde com os termos de uso e política de privacidade

# Melhoria no método de busca

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='mmr', #algoritmo relevância marginal máxima que vai selecionar os documentos recuperados principalmente para evitar dados duplicados,
    #garantindo um equilíbrio entre relevância e diversidade dos itens recuperados por relevância e diversidade
    search_kwargs={'k':3, 'fetch_k':4}  #fetch_k: quantos documentos são recuperados antes de aplicar o algoritmo nmr
)

Valores maiores de fetch-k, entre 20 e 50, podem gerar resultados com maior relevância, pois o algoritmo terá mais documentos para escolher, porém, há maior custo computacional

Valores ideais para fetch_k dependem do tamanho e qualidade da base de dados e do equilíbrio desejado entre qualidade do resultado e desempenho.

Para este estudo de caso, faremos:

Valores de 1 até 5: quando se precisa de poucos documentos altamente relevantes, como em tarefas simples de perguntas e respostas

Valores de 10 até 20: para quando se quer oferecer mais contexto ou opções ao usuário, como em um mecanismo de busca ou sistema de recomendação.

Portanto, os melhores valores dependem do caso de uso que está sendo implementado.

# Pipeline RAG